# Importação das Bibliotecas

In [30]:
import cv2
import json
import numpy as np
import os

# Captação das Vagas

In [40]:
# leitura da imagem
while True:
    entrada = input('Nome do arquivo (com extensão, ex: imagem.png): ') # usuário deve inserir o nome do arquivo
    
    if entrada == 'x':
        print("Encerrado")
        break

    if os.path.isfile(entrada):
        img = cv2.imread(entrada)
        if img is not None:
            break
        else:
            print("Erro ao abrir a imagem. Tente outro arquivo.")
    else:
        print("Arquivo não encontrado. Tenta de novo.")

vaga_atual = []
todas_vagas = []

# função para selecionar as vagas
def clique(event, x, y, flags, param):
    global vaga_atual, todas_vagas

    if event == cv2.EVENT_LBUTTONDOWN: # se o botão esquerdo for pressionado
        vaga_atual.append((x, y)) # adiciona o ponto à lista da vaga atual
        # Desenhar o ponto onde o usuário clicou (círculo azul)
        cv2.circle(img_clone, (x, y), 5, (255, 0, 0), -1)  # desenha um círculo azul onde o usuário clicou (extremidade da vaga)
        print(f"Ponto clicado: {(x, y)}") # printa o ponto clicado

    if len(vaga_atual) == 2: # se o usuário já fez 2 cliques (preencheu uma vaga):
        todas_vagas.append(vaga_atual.copy()) # salva a vaga na lista de vagas, que contém as coordenadas de cada vaga
        cv2.rectangle(img_clone, vaga_atual[0], vaga_atual[1], (255, 0, 0), 2) # desenha um retângulo azul, mostrando onde a vaga foi selecionada
        print(f"Vaga salva: {vaga_atual}") # printa os pontos da vaga, mostrando que ela foi lida
        vaga_atual = [] # reseta a lista da vaga atual para a próxima poder ser armazenada


img = cv2.imread(entrada) # lê a imagem que o usuário digitou
img_clone = img.copy() # cria uma cópia da imagem original, que será usada para desenhar as vagas
cv2.namedWindow("Imagem") # cria uma janela onde a imagem será exibida
cv2.setMouseCallback("Imagem", clique) # define a função 'clique' para ser chamada quando o usuário clicar na janela

while True:
    cv2.imshow("Imagem", img_clone) # mostra a imagem
    key = cv2.waitKey(1)
    if key == 27:
        break # sai do loop se ESC for pressionado

cv2.destroyAllWindows() # fecha a janela aberta pelo opencv

with open("coordenadas_vagas.json", "w") as f:
    json.dump(todas_vagas, f) # salva as coordenadas em um json

Ponto clicado: (233, 20)
Ponto clicado: (264, 95)
Vaga salva: [(233, 20), (264, 95)]
Ponto clicado: (217, 138)
Ponto clicado: (255, 222)
Vaga salva: [(217, 138), (255, 222)]
Ponto clicado: (291, 221)
Ponto clicado: (321, 294)
Vaga salva: [(291, 221), (321, 294)]
Ponto clicado: (320, 153)
Ponto clicado: (353, 220)
Vaga salva: [(320, 153), (353, 220)]
Ponto clicado: (425, 223)
Ponto clicado: (465, 292)
Vaga salva: [(425, 223), (465, 292)]
Ponto clicado: (492, 226)
Ponto clicado: (535, 297)
Vaga salva: [(492, 226), (535, 297)]
Ponto clicado: (172, 18)
Ponto clicado: (202, 88)
Vaga salva: [(172, 18), (202, 88)]
Ponto clicado: (329, 22)
Ponto clicado: (359, 92)
Vaga salva: [(329, 22), (359, 92)]
Ponto clicado: (415, 22)
Ponto clicado: (455, 100)
Vaga salva: [(415, 22), (455, 100)]
Ponto clicado: (125, 151)
Ponto clicado: (149, 220)
Vaga salva: [(125, 151), (149, 220)]
Ponto clicado: (189, 222)
Ponto clicado: (217, 290)
Vaga salva: [(189, 222), (217, 290)]
Ponto clicado: (386, 148)
Ponto cli

# Interpretação das Vagas (utilizando média)

In [44]:
img = cv2.imread(entrada)  # lê a imagem
with open("coordenadas_vagas.json", "r") as f:
    vagas = json.load(f)  # lê o json com as coordenadas das vagas definidas

livres = 0
ocupadas = 0
entropias = []

def calcular_entropia(imagem):
    histograma = cv2.calcHist([imagem], [0], None, [256], [0, 256])
    histograma = histograma / histograma.sum()
    entropia = -np.sum(histograma * np.log2(histograma + 1e-7))
    return entropia

# Primeiro loop: coleta as entropias
for vaga in vagas:
    (x1, y1), (x2, y2) = vaga
    vaga_crop = img[y1:y2, x1:x2]
    cinza = cv2.cvtColor(vaga_crop, cv2.COLOR_BGR2GRAY)
    entropias.append(calcular_entropia(cinza))

# Define o limiar com base na média + deslocamento
limiar = np.mean(entropias)
print(f"Limiar baseado na média das entropias: {limiar:.2f}")

# Segundo loop: classifica com base no limiar adaptativo
for i, vaga in enumerate(vagas):
    (x1, y1), (x2, y2) = vaga
    vaga_crop = img[y1:y2, x1:x2]
    cinza = cv2.cvtColor(vaga_crop, cv2.COLOR_BGR2GRAY)
    entropia = calcular_entropia(cinza)
    print(f"Vaga {i+1}: entropia = {entropia:.2f}")

    if entropia < limiar:
        cor = (0, 255, 0)
        livres += 1
    else:
        cor = (0, 0, 255)
        ocupadas += 1

    cv2.rectangle(img, (x1, y1), (x2, y2), cor, 2)

cv2.imshow("Resultado", img)
cv2.imwrite("resultado.png", img)

while True:
    cv2.imshow("Resultado", img) # mostra a imagem
    key = cv2.waitKey(1)
    if key == 27:
        break

cv2.destroyAllWindows()

Limiar baseado na média das entropias: 6.39
Vaga 1: entropia = 5.59
Vaga 2: entropia = 5.36
Vaga 3: entropia = 5.45
Vaga 4: entropia = 7.17
Vaga 5: entropia = 7.30
Vaga 6: entropia = 7.08
Vaga 7: entropia = 6.84
Vaga 8: entropia = 7.20
Vaga 9: entropia = 7.04
Vaga 10: entropia = 6.57
Vaga 11: entropia = 5.31
Vaga 12: entropia = 6.14
Vaga 13: entropia = 6.14
Vaga 14: entropia = 5.27
Vaga 15: entropia = 7.18


# Relatório

In [42]:
relatorio = []

# escreve no relatório
relatorio.append(f"Total de vagas: {livres + ocupadas}")
relatorio.append(f"Vagas livres: {livres}")
relatorio.append(f"Vagas ocupadas: {ocupadas}")

# salva o relatório
with open("relatorio_vagas.txt", "w") as f:
    f.write("\n".join(relatorio))

# Foto com Legenda

In [43]:
img_legenda = cv2.imread("resultado.png")

total = livres + ocupadas
texto = [
    f"Total: {total}",
    f"Livres: {livres}",
    f"Ocupadas: {ocupadas}"
]

# fundo branco
overlay = img_legenda.copy()
cv2.rectangle(overlay, (10, img_legenda.shape[0]-50), (220, img_legenda.shape[0]-10), (255, 255, 255), -1)
img_legenda = cv2.addWeighted(overlay, 0.6, img_legenda, 0.4, 0)

# escrever texto com fonte menor
for i, linha in enumerate(texto):
    y = img_legenda.shape[0] - 35 + i*15
    cv2.putText(img_legenda, linha, (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

cv2.imwrite("legenda.png", img_legenda)


True